# Ag Analytics-PolarisSoilsDataLayers-Demo


In [ ]:
import requests
import json
import time
from pandas.io.json import json_normalize
from collections import defaultdict
import pandas as pd
import geopandas as gpd
import requests, zipfile, io
%matplotlib inline
%autosave 0

### API Details:

####Request URL: https://ag-analytics.azure-api.net/PolarisSoilsDataLayers/get[?inputShape][&SoilValue][&Statistics][&SoilHorizon]

####Parameters:

***inputShape - ***
The shape information for field in esriGeometryPolygon format.

***SoilValue - ***
e.g. pH, Organic Matter

***Statistics - ***
Statistics options are: min, max, mean, and variance

***SoilHorizon - ***
Soil depth in CM.


In order to get Ocp-Apim-Subscription-Key, please click on this link https://ag-analytics.org/developer/Session/SignInFromPayment

### Request Parameters

In [ ]:
inputShape={"geometryType":"esriGeometryPolygon","features":[{"geometry":{"rings":[[[-89.045774,43.730605],[-89.04577,43.730488],[-89.045722,43.729998],[-89.045789,43.729653],[-89.045661,43.72961],[-89.045635,43.729782],[-89.045644,43.730045],[-89.045653,43.730308],[-89.045651,43.730607],[-89.041228,43.730676],[-89.041171,43.728908],[-89.041111,43.727076],[-89.042892,43.727044],[-89.044665,43.727018],[-89.046178,43.726999],[-89.046237,43.730597],[-89.045774,43.730605]]],"spatialReference":{"wkid":4326}}}]}
SoilValue="ph"
Statistics="mean"
SoilHorizon="0_5"
headers={'Ocp-Apim-Subscription-Key': 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'}
           

###  API Function

In [ ]:
def get_polarissoil_datalayers(inputShape,SoilValue,Statistics,SoilHorizon,headers):
    try:
        url = "https://ag-analytics.azure-api.net/PolarisSoilsDataLayers/get?inputShape={}&SoilValue={}&Statistics={}&SoilHorizon={}".format(inputShape,SoilValue,Statistics,SoilHorizon)
     
        response = (requests.get(url,headers=headers).text)
        
        response_json = json.loads(response)
        #print(response_json)
        time.sleep(0.5)
        return response_json
    
    except Exception as e:
        raise e

### Calling API Function and Displaying Response

In [ ]:
polaris=get_polarissoil_datalayers(inputShape,SoilValue,Statistics,SoilHorizon,headers)

### Displaying Table

In [ ]:
df=defaultdict(list)
resolution=5
areaOfOnePixel=resolution*resolution
totalpixels = 0
for value in polaris['results'][3]['value']: 
    val=str(round(value[0] * 100) / 100) + "-" + str(round(value[1] * 100) / 100)
    df[SoilValue].append(val)
for value in polaris['results'][2]['value']['features']: 
    totalpixels+=value['attributes']['Count']

totalArea=(totalpixels * areaOfOnePixel) * 0.000247105

for value in polaris['results'][2]['value']['features']: 
    partAcres = (value['attributes']['Count'] * areaOfOnePixel) * 0.000247105
    percentAcres = (partAcres / totalArea) * 100
    df['Area per Part (Acres)'].append(round(partAcres * 100) / 100)
    df['% of Total'].append(round(percentAcres * 100) / 100)

soil=pd.DataFrame.from_dict(df)
soil

### Displaying  Map

#### Helper Functions

In [ ]:
def post_converttogeojson(requestbody):
    try:
        url = "https://ogre.adc4gis.com/convert"
     
        response = (requests.post(url,files=requestbody).text)
        
        return response

    except Exception as e:
        raise e
def post_converttoshapefile(requestbody):
    try:
        url = "https://ogre.adc4gis.com/convertJson"
     
        response = (requests.post(url,data=requestbody))
        
        z = zipfile.ZipFile(io.BytesIO(response.content))
        z.extractall("./shapefiles/")
        
        time.sleep(0.5)
        return response
    
    except Exception as e:
        raise e

In [ ]:
requestbody={'upload':open('./jsonfiles/polarisdata1.json','rb')}

polarisdata_geojson=post_converttogeojson(requestbody)
requestbody={'json':polarisdata_geojson,'skipFailures':"on"}
post_converttoshapefile(requestbody)

#### Displaying Output Map

In [ ]:
gdf = gpd.read_file('./shapefiles/OGRGeoJSON.shp')
gdf.plot(column='val_float', cmap=None,figsize=(10, 10),legend=True) 
gdf.head()